In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)

css_styling()

In [2]:
from torch import max, no_grad
from torch.nn import Conv2d, MaxPool2d, ReLU, Softmax, Flatten, Module, Linear, CrossEntropyLoss
from torch.nn import functional as f
from torch.optim import Adam, SGD
from torchvision.transforms import Compose, Normalize, ToTensor
from torchvision.datasets import FashionMNIST

In [3]:
transform = Compose([
    ToTensor(),
    Normalize((0.5, ), (0.5, )),
])

In [ ]:
train = FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

test = FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform,
)

In [5]:
from torch.utils.data import DataLoader

In [6]:
trainloader = DataLoader(
    train,
    batch_size=32,
    shuffle=True,
    num_workers=0,
)
testloader = DataLoader(
    test,
    batch_size=32,
    shuffle=False,
    num_workers=0,
)

In [7]:
 class ConvNet(Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv_1 = Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv_2 = Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.maxpool = MaxPool2d(kernel_size=2, stride=2)
        self.fc_1 = Linear(64 * 12 * 12, 128)
        self.fc_2 = Linear(128, 10)
        
    def forward(self, x):
        x = f.relu(self.conv_1(x))
        x = self.maxpool(f.relu(self.conv_2(x)))
        x = x.view(-1, 64*12*12)  # Flatten Layer
        x = f.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x

In [8]:
covnet = ConvNet()

In [9]:
covnet.to("cpu")

ConvNet(
  (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_1): Linear(in_features=9216, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
loss_fn = CrossEntropyLoss()
optimizer = SGD(covnet.parameters(), lr=0.001, momentum=0.9)

In [12]:
epochs = 8



def train_model(model, trainloader, testloader, loss_fn, optimizer, epochs=5):
    epoch_log = []
    loss_log = []
    acc_log = []

    for epoch in range(epochs):
        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to("cpu")
            labels = labels.to("cpu")

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()  # Back propagate to obtain the new gradients for all nodes
            optimizer.step() # Update the gradients/weights

            running_loss += loss.item()

            if i % 50 == 49:
                correct = 0
                total = 0

                with no_grad():
                    for data in testloader:
                        images, labels = data
                        images = images.to("cpu")
                        labels = labels.to("cpu")

                        outputs = covnet(images)

                        _, predicted = max(outputs.data, dim=1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                    acc = 100 * correct / total
                    actual_loss = running_loss / 50.0
                    print(f"""Epoch: {(epoch+1)}
                    Minibatches completed: {(i+1)} 
                    Loss: {actual_loss:.3f} 
                    Test acc: {acc:.3f}
                    """)

                    running_loss = 0.0

        epoch_log.append((epoch+1))
        loss_log.append(actual_loss)
        acc_log.append(acc)
    
    return (epoch_log, loss_log, acc_log)

In [13]:
epoch_log, loss_log, acc_log = train_model(covnet, trainloader, testloader, loss_fn, optimizer, epochs)

Epoch: 1
                    Minibatches completed: 50 
                    Loss: 2.195 
                    Test acc: 48.470
                    
Epoch: 1
                    Minibatches completed: 100 
                    Loss: 1.563 
                    Test acc: 64.130
                    
Epoch: 1
                    Minibatches completed: 150 
                    Loss: 0.921 
                    Test acc: 69.220
                    
Epoch: 1
                    Minibatches completed: 200 
                    Loss: 0.792 
                    Test acc: 67.180
                    
Epoch: 1
                    Minibatches completed: 250 
                    Loss: 0.723 
                    Test acc: 73.300
                    
Epoch: 1
                    Minibatches completed: 300 
                    Loss: 0.711 
                    Test acc: 71.640
                    
Epoch: 1
                    Minibatches completed: 350 
                    Loss: 0.693 
                    Tes

Epoch: 2
                    Minibatches completed: 1000 
                    Loss: 0.438 
                    Test acc: 84.070
                    
Epoch: 2
                    Minibatches completed: 1050 
                    Loss: 0.460 
                    Test acc: 84.160
                    
Epoch: 2
                    Minibatches completed: 1100 
                    Loss: 0.398 
                    Test acc: 84.110
                    
Epoch: 2
                    Minibatches completed: 1150 
                    Loss: 0.420 
                    Test acc: 84.890
                    
Epoch: 2
                    Minibatches completed: 1200 
                    Loss: 0.413 
                    Test acc: 84.670
                    
Epoch: 2
                    Minibatches completed: 1250 
                    Loss: 0.397 
                    Test acc: 81.090
                    
Epoch: 2
                    Minibatches completed: 1300 
                    Loss: 0.397 
               

Epoch: 4
                    Minibatches completed: 100 
                    Loss: 0.356 
                    Test acc: 87.030
                    
Epoch: 4
                    Minibatches completed: 150 
                    Loss: 0.363 
                    Test acc: 86.300
                    
Epoch: 4
                    Minibatches completed: 200 
                    Loss: 0.309 
                    Test acc: 86.820
                    
Epoch: 4
                    Minibatches completed: 250 
                    Loss: 0.350 
                    Test acc: 87.300
                    
Epoch: 4
                    Minibatches completed: 300 
                    Loss: 0.353 
                    Test acc: 87.190
                    
Epoch: 4
                    Minibatches completed: 350 
                    Loss: 0.298 
                    Test acc: 86.770
                    
Epoch: 4
                    Minibatches completed: 400 
                    Loss: 0.317 
                    Te

Epoch: 5
                    Minibatches completed: 1050 
                    Loss: 0.290 
                    Test acc: 88.620
                    
Epoch: 5
                    Minibatches completed: 1100 
                    Loss: 0.272 
                    Test acc: 87.900
                    
Epoch: 5
                    Minibatches completed: 1150 
                    Loss: 0.295 
                    Test acc: 87.890
                    
Epoch: 5
                    Minibatches completed: 1200 
                    Loss: 0.297 
                    Test acc: 88.000
                    
Epoch: 5
                    Minibatches completed: 1250 
                    Loss: 0.280 
                    Test acc: 88.430
                    
Epoch: 5
                    Minibatches completed: 1300 
                    Loss: 0.267 
                    Test acc: 88.300
                    
Epoch: 5
                    Minibatches completed: 1350 
                    Loss: 0.298 
               

Epoch: 7
                    Minibatches completed: 150 
                    Loss: 0.275 
                    Test acc: 88.850
                    
Epoch: 7
                    Minibatches completed: 200 
                    Loss: 0.265 
                    Test acc: 89.400
                    
Epoch: 7
                    Minibatches completed: 250 
                    Loss: 0.237 
                    Test acc: 89.140
                    
Epoch: 7
                    Minibatches completed: 300 
                    Loss: 0.263 
                    Test acc: 88.950
                    
Epoch: 7
                    Minibatches completed: 350 
                    Loss: 0.238 
                    Test acc: 88.660
                    
Epoch: 7
                    Minibatches completed: 400 
                    Loss: 0.272 
                    Test acc: 89.270
                    
Epoch: 7
                    Minibatches completed: 450 
                    Loss: 0.240 
                    Te

Epoch: 8
                    Minibatches completed: 1100 
                    Loss: 0.249 
                    Test acc: 89.430
                    
Epoch: 8
                    Minibatches completed: 1150 
                    Loss: 0.237 
                    Test acc: 89.720
                    
Epoch: 8
                    Minibatches completed: 1200 
                    Loss: 0.258 
                    Test acc: 89.500
                    
Epoch: 8
                    Minibatches completed: 1250 
                    Loss: 0.217 
                    Test acc: 89.310
                    
Epoch: 8
                    Minibatches completed: 1300 
                    Loss: 0.245 
                    Test acc: 89.810
                    
Epoch: 8
                    Minibatches completed: 1350 
                    Loss: 0.201 
                    Test acc: 89.740
                    
Epoch: 8
                    Minibatches completed: 1400 
                    Loss: 0.238 
               